## CHAPTER 5 - 순환 신경망(RNN)


#### 5.1 확률과 언어 모델
---
이제 우리는 단어의 분산 표현을 얻었으니 이를 응용하는 모델을 만들어야 한다.

CBOW 모델을 학습시키는 본래 목적은 맥락으로부터 타깃을 정확하게 추측하는 것이다. 이 목적을 위해 학습을 진행하면, (그 부산물로) 단어의 의미가 인코딩된 '단어의 분산 표현'을 얻을 수 있다. 

CBOW 모델을 언어 모델로 사용하기엔 무리가 있는게...

CBOW 모델은 맥락 안의 단어 순서를 무시한체 타깃을 추출한다. 즉, (you, say)나 (say, you)나 같은 걸로 판단한다는 것

![](../image/image_40.png)

#### 5.2 RNN이란
---
RNN (Recurrent Neural Network) 순환 신경망

![](../image/image_41.png)
![](../image/image_42.png)
![](../image/image_43.png)
![](../image/image_44.png)

즉 간단하게 말하자면

우리가 말을 할때에도 시간을 기준으로 차차 단어가 나온다

에초에 이 글을 쓸 때에도

나는 매 시간에 따라서 단어를 조합해서 입력하지 않는가??

말을 할 때 어 음 내가 무슨 말을 해야하지 라고 생각하고 모든 단어를 생각해서 0.1초에 동시에 딱 말하지 는 않는다 컴퓨터도 아니고…..

즉 시간의 관점에서 우리가 말을 빠르게 해봐야 시간에 따라서 단어가 나열될 것이고 그 단어의 순서는 반드시 단어의 의미를 내포하고 있을 것이란 거지 분포 가설처럼 ‘단어의 의미는 주변 단어에 의해 형성된다.’

는 가설처럼 단어의 의미는 주변 단어에 의해 형성되고 그 단어들은 시간에 따라서 정렬되었다는 의미이다

우리가 글을 쓸 때에도 왼쪽에서 오른쪽으로 시간이 흐르면서 쓰지 않는가

##### 5.2.1 순환 구조 펼치기
![](../image/image_45.png)
![](../image/image_46.png)

##### 5.2.2 BPTT
![](../image/image_47.png)

![](../image/image_48.png)

![](../image/image_49.png)

![](../image/image_50.png)


이렇게 역전파가 끊어짐에도 불구하고 학습이 잘 되는 이유를 알려면 가중치들이 갱신되는 원리와 학습 과정 일부를 이해한다면 알 수 있습니다.

위의 RNN 계층 그림도 사실 하나의 계층에 대해서 시간적으로 나타낸 것이기 때문에 본질이 하나이므로 가중치들(Wx, Wh, b)은 모두 같아야 합니다.

하지만 Truncated 기법을 도입하면서 역전파가 끊어지게 되므로 가중치 조건에서 어느 정도 변화가 생깁니다.


그 변화는 같은 블럭 내부의 계층들 마다의 가중치 조건은 모두 같지만 서로 다른 블럭들의 가중치 조건은 다르다는 것입니다.

(여기서 가중치 조건이랑 가중치 값 그 자체를 말함)

가중치 갱신이 다음과 같이 일어나기 때문입니다.

가중치 갱신은 첫 번째 블록이 블록 내부에서 순전파를 실행하고 역전파 과정을 거쳐서 기울기를 바탕으로 갱신 된 가중치 조건들이 두 번째 블록의 가중치 초기 조건으로 사용됩니다.

마찬가지로 두 번째 블록의 계층들이 해당 초기 가중치 조건으로 순전파를 진행한 후 다시 끝에서 처음까지(블록 안 기준) 역전파를 해서 갱신된 가중치 조건들이 세 번째 블록의 가중치 초기 조건으로 사용됩니다.

이런식으로 반복이되고 이를 통해 기존 블록의 갱신된 가중치가 다음 블록으로 넘어가서 실행되고 갱신되므로 역전파를 끊어서 사용해도 학습에 크게 지장이 없다는 것을 알 수 있습니다.

그래서 Truncated BPTT는 일반 BPTT에서 역전파 부분을 끊어주어서 계산비용을 줄여주고, 과거에 대한 정보를 기억하기 위해서 순전파는 유지를 시켜주는 형태로 구성된 것입니다.


##### 5.2.4 Truncated BPTT의 미니배치 학습
![](../image/image_51.png)
![](../image/image_52.png)

#### 5.3 RNN 구현
---
![](../image/image_53.png)

![](../image/image_54.png)


##### 5.3.1 파이토치의 RNN
> https://pytorch.org/docs/stable/generated/torch.nn.RNN.html#torch.nn.RNN

torch.nn.RNN()

1. 파라미터

    1. input_size 

        입력해 주는 특성 값의 개수 입니다. 만약 feature의 개수가 1이라면 input_size=1, 입력 feature 개수가 7개면 input_size=7을 입력합니다.

    2. hidden_size

        hidden state의 개수를 지정합니다. 보통 arbitrary 합니다.

    3. num_layers

        RNN 레이어를 겹겹이 쌓아올릴 수 있습니다. RNN 레이어를 쌓아 올리는 것을 stacking RNN이라고도 합니다. 만약, 2개층을 겹겹이 쌓아올린다면 num_layers=2 로 설정하면 됩니다. 기본 값: 1

    4. nonlinearity

        사용할 비선형성입니다. 'tanh' 또는 'relu'일 수 있습니다. 기본값: 'tanh'

    5. bias

        False인 경우 레이어는 바이어스 가중치 b_ih와 b_hh를 사용하지 않습니다. 기본값: True

    6. batch_first

        입력으로 받는 데이터의 shape중 첫 번째 차원을 batch로 간주할 것인지를 설정합니다. 일반적으로 pytorch에서 데이터 전처리시 batch를 첫번째 차원으로 지정하기 때문에 많은 케이스에서 batch_firtst=True 로 지정함을 볼 수 있습니다. True이면 입력 및 출력 텐서가 (seq, batch, feature) 대신 (batch, seq, feature)로 제공됩니다. 이는 hidden 또는 cell state에는 적용되지 않습니다. 기본값: False

    7. dropout

        0이 아닌 경우, 마지막 레이어를 제외한 각 RNN 레이어의 출력에 드롭아웃 레이어를 도입하며, 드롭아웃 확률은 dropout과 같습니다. 기본값: 0
        
    8. bidirectional
    
        True이면 양방향 RNN이 됩니다. 기본값: False

2. 사용 예시
``` py
>>> rnn = nn.RNN(10, 20, 2)
>>> input = torch.randn(5, 3, 10)
>>> h0 = torch.randn(2, 3, 20)
>>> output, hn = rnn(input, h0)
```




#### 5.3 시계열 데이터 처리 계층 구현
---
![](../image/image_56.png)

첫 번째 층은 Embedding계층으로 이 계층은 단어 ID를 단어의 분산 표현(단어 벡터)으로 변환한다.

그리고 그 분산 표현이 RNN 계층으로 입력된다.

![](../image/image_57.png)

첫 단어로 단어 ID가 0인 “you”가 입력됩니다. 이때 Softmax 계층이 출력하는 확률분포를 보면 ‘say’에서 가장 높게 나온 것을 알 수 있습니다. 

다시 말해서 ‘you’ 다음에 출현하는 단어가 “say”라는 것을 올바르게 예측했습니다. 

당연하지만 이처럼 제대로 예측하려면 좋은 가중치(잘 학습된 가중치)를 사용해야 합니다.

두 번째 단어인 “say”를 입력하는 부분에 주목해봅시다. 이떄의 softmax 계층 출력은 “goodbye”와 “hello”, 두곳에서 높게 나왔습니다. 

확실히 “you say goodbye”와 “you say hello”는 모두 자연스러운 문장입니다. 

여기서 주목할 점은 RNN 계층은 “you say”라는 맥락을 ‘기억’하고 있다는 사실입니다. 

더 정확하게 말하자면, RNN은 “you say”라는 과거의 정보를 응집된 은닉 상태 벡터로 저장해두고 있습니다.

RNNLM은 지금까지 입력된 단어를 ‘기억’하고 그것을 바탕으로 다음에 출현한 단어를 예측합니다.

이 일을 가능하게 하는 비결이 바로 RNN 계층의 존재입니다. RNN 계층이 과거에서 현재로 데이터를 계속 흘려보내줌으로써 과거의 정보를 인코딩해 저장(기억)할 수 있는 것이죠

![](../image/image_58.png)

![](../image/image_59.png)

#### 5.5 RNNLM 학습과 평가
---

![](../image/image_60.png)

### 여담
---
미니배치가 나와서 헷갈리니 딥러닝 개념을 다시 올려보겠다.

![](../image/image_61.png)